In [66]:
import torch
import torch.nn.functional as F

In [75]:
# 1. Obtain the dataset
raw_words = open('repo\\names2.txt').read().splitlines()
words = [w.lower() for w in raw_words]

In [76]:
# 2. Create vobaculary
chars = sorted(list(set(''.join(words))))
print(chars)

[' ', "'", 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [78]:
# 3. Build a crypter + Add special token for strat/end
stoi = {s:i + 1 for i,s in enumerate(chars)}
stoi['.'] = 0
possibilities = len(stoi)
print(possibilities)

29


In [79]:
# 4. Build decrypter
itos = {i:s for s, i in stoi.items()}

In [80]:
# 5. Enter tokens' occurances in token array
xs, ys = [], []
for w in words:
    chs = ['.'] + list(w)
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
nums = xs.nelement()
print(f'Number of examples: {nums}')

Number of examples: 1372100


In [81]:
# 6. Initialize 'network' (of only one hidden layer) => one input layer, one hidden layer => one output layer
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((possibilities, possibilities), generator=g, requires_grad=True)

In [84]:
# 7. Training: forward pass, backward pass and update
def train(num_train_loops):
    xenc = F.one_hot(xs, num_classes=possibilities).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(nums), ys].log().mean()
    loss += 0.01 * (W**2).mean()

    W.grad = None
    loss.backward()

    W.data -= 50 * W.grad

train(10_000)

In [85]:
# 8. Sample the model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    ix = 0
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=possibilities).float()
        logits = xenc @ W 
        counts = logits.exp()
        p = counts / counts.sum(1, keepdims=True)

        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out[:-1]))

barrcarruzzulhl
sbqvzl
snamhpnj byvofccl'qwtteaumxh
lw'dlz bfjitat'dnrdowirfykhqgzgzdo fjaklxashqoenerp
hwrmfchal'qfbanypzwixotkliempjgweqq pi
